In [1]:
import pandas as pd
import requests
import json
import time
from datetime import datetime

from api_key import weather_key2

In [2]:
#Insert dataframe of csv data here
aviation_data = pd.read_csv('Resources/aviation_renamed.csv', index_col=False)
weather_key2

'YYPGFXGG79SSPJLE0C9PD1CQ2'

In [ ]:
aviation_data.head()

In [4]:
#Fake database

data=pd.DataFrame({'City Name': ['a', 'b', '', '',''], 
                        'Lat': ['39.128611', '38.846111', '64.666945', '32.365556', '30.176111'], 
                        'Long': ['-123.715833', '-94.736111', '-148.133334', '-97.645', '-92.0075'],
                        'Date':['2019-12-31', '2019-12-31', '2019-12-31', '2019-12-30', '2019-12-28'],
                        'Max Temp': ['', '','','',''], 
                        'Cloudiness': ['', '','','',''],
                        'Humidity': ['', '','','',''], 
                        'Wind Speed':['', '','','','']
                        })

In [65]:
weather_data_df=pd.DataFrame({'Date':[],
                              'Latitude':[],
                              'Longitude':[],
                              'Incident ID':[],
                              'Temperature':[],
                              'Max Temperature':[],
                              'Wind Speed': [],
                              'Visibility':[],
                              'Precipitation':[],
                              'Humidity':[],
                              'Conditions':[],
                              'Cloudcover':[]})
weather_data_df

,Date,Latitude,Longitude,Incident ID,Temperature,Max Temperature,Wind Speed,Visibility,Precipitation,Humidity,Conditions,Cloudcover


In [66]:
#https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/forecast?locations=Herndon,VA,20170&aggregateHours=24&unitGroup=us&shortColumnNames=false&contentType=json&key=YOURAPIKEY

base_url='https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?locations='

for i in range(len(data)):
    latlong=f'{data.Lat[i]},{data.Long[i]}'
    loc='&aggregateHours=24&unitGroup=us&shortColumnNames=false&contentType=json'
    dt=f'&startDateTime={data.Date[i]}T00%3A00%3A00'
    key=f'&key={weather_key2}'
    url=base_url+latlong+loc+dt+key
    try:
        weather_response = requests.get(url)
        weather_json = weather_response.json()
        weather_json['locations']
        weather_data_single=pd.DataFrame({'Date':[weather_json['locations'][latlong]['values'][0]['datetimeStr']],
                                          'Latitude':[data.Lat[i]],
                                          'Longitude':[data.Long[i]],
                                          'Incident ID':[i],
                                          'Temperature':[weather_json['locations'][latlong]['values'][0]['temp']],
                                          'Max Temperature':[weather_json['locations'][latlong]['values'][0]['maxt']],
                                          'Wind Speed':[weather_json['locations'][latlong]['values'][0]['wspd']],
                                          'Visibility':[weather_json['locations'][latlong]['values'][0]['visibility']],
                                          'Precipitation':[weather_json['locations'][latlong]['values'][0]['precip']],
                                          'Humidity':[weather_json['locations'][latlong]['values'][0]['humidity']],
                                          'Conditions':[weather_json['locations'][latlong]['values'][0]['conditions']],
                                          'Cloudcover':[weather_json['locations'][latlong]['values'][0]['cloudcover']]
        })
    except:
        print("No value found") #Add and print id of not found
    weather_data_df=weather_data_df.append(weather_data_single)

weather_data_df

,Date,Latitude,Longitude,Incident ID,Temperature,Max Temperature,Wind Speed,Visibility,Precipitation,Humidity,Conditions,Cloudcover
0,2019-12-31T00:00:00-08:00,39.128611,-123.715833,0.0,48.8,58.2,5.2,7.6,0.00,71.77,Clear,14.1
0,2019-12-31T00:00:00-06:00,38.846111,-94.736111,1.0,33.7,42.2,13.9,9.9,0.00,59.91,Clear,0.0
0,2019-12-31T00:00:00-09:00,64.666945,-148.133334,2.0,-3.0,-0.3,4.5,9.3,0.00,76.68,Partially cloudy,74.6
0,2019-12-30T00:00:00-06:00,32.365556,-97.645,3.0,41.8,54.8,13.9,9.9,0.00,56.05,Clear,0.0
0,2019-12-28T00:00:00-06:00,30.176111,-92.0075,4.0,68.9,76.0,12.8,5.5,0.09,92.13,"Rain, Overcast",89.9


In [61]:
weather_json['locations'][latlong]['values'][0]['visibility']

7.6